In [ ]:
import json
import os
import requests
from io import BytesIO
from PIL import Image
from tqdm import tqdm

jsonl_path = "all_vsr_raw_data.jsonl"
out_root = "vsr_sd_full"

os.makedirs(f"{out_root}/images", exist_ok=True)
cap_file = open(f"{out_root}/captions.txt", "w", encoding="utf-8")

idx = 0

count = 0
for line in tqdm(open(jsonl_path, "r")):
    obj = json.loads(line)

    if obj["label"] != 1:
        continue

    url = obj["image_link"]
    caption = obj["caption"]

    # download
    try:
        resp = requests.get(url, timeout=10)
        img = Image.open(BytesIO(resp.content)).convert("RGB")
    except Exception as e:
        print("Failed:", url, e)
        continue

    img = img.resize((512, 512), Image.BICUBIC)
    
    out_name = f"{idx:05d}.png"
    img.save(f"{out_root}/images/{out_name}")

    cap_file.write(f"{out_name}\t{caption}\n")
    idx += 1

cap_file.close()
print("Done! Total images:", idx)

12809it [22:45,  9.38it/s]

Done! Total images: 6293
